In [2]:
import matplotlib.pyplot as plt
import numpy as np
import time
import os
# from scipy.special import kv
# from scipy.integrate import quad
from importlib import reload
import pickle
from scipy.stats import maxwell

from pulsars import Pulsars, c, e_charge, e_mass, h
import speiser_fun as sf 
import speiser_fun_cyl as sfc
import speiser_integ as si
import speiser_plots as sp
import speiser_model as sm
import speiser_spec as ss

In [3]:
k = 500.
name = 'crab'
pulsar = Pulsars(k)[name]

gamma1 = 1.
B_0 = pulsar['Blc']
omegaB = (e_charge*B_0/(e_mass*c))    #γυροσυχνότητα
Rlc = pulsar['rlc']*omegaB/c
Delta = pulsar['rlc']/k*omegaB/c
# q = 1.

gamma0 = maxwell.rvs(loc = 0, scale = 350, size=10)

init_pos = np.linspace(0.01, 1., 10)

delta_init = maxwell.stats(loc = 0, scale = 350, moments = 'm')    #πάχος του φύλλου ρεύματος, αδιάστατο, σε μονάδες [c/ωΒ]

# delta = 500.
# gamma0 = np.array([1000., 1.])   #αρχικός παράγοντας Lorentz
# init_pos = np.array([1.])


T = 150.
N = 10**5
t = np.linspace(0.0, T*Delta, N)


In [3]:
#Δημιουργία λεξικού με τις παραμέτους της προσωμοίωσης σε αναγνώσιμη και δυαδική μορφή
parameters = {'pulsar_name':name, 'char magn field (Blc)':B_0, 'Radius of lc':Rlc, 'multiplicity':k, 
              'diss zone length':Delta, 'diss zone width':delta_init, 'runtime':T, 'number of t points':N, 
              'initial lorentz factors':gamma0, 'initial injection positions':init_pos}

dir_name = 'data'

#Δημιουργεία φακέλου για την αποθηκευση των δεδομένων
for i in range(1, 1000):
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)
        print('directory ', dir_name, 'created')
        break
    else:
        dir_name = 'data{}'.format(i)
        
with open('{}/parameters.txt'.format(dir_name), 'w') as par_file:
    for key, val in parameters.items():
        par_file.write('{} --> {}\n'.format(key, val))
with open('{}/par_dic.txt'.format(dir_name), 'wb') as par_file:
    pickle.dump(parameters, par_file)

directory  data7 created


In [4]:
for q in [-1, 1]:
    if q == +1:
        charge = 'pos'
    else:
        charge = 'el'
    
    print(charge)
    
    for w in range(len(init_pos)):

        print(init_pos[w])

        (r, ur, phi, uphi, z_cyl, uz_cyl), dic_cyl = si.oloklirosi(gamma0, Rlc, Delta, delta_init, B_0, t, init_pos[w], q, T, 
                                                                   sm.speiser_cyl, coord = 'cyl')
            
        with open('{}/cyl_{}_{}_raw.npz'.format(dir_name, int(100*init_pos[w]), charge), 'wb') as f:
            np.savez(f, r = r, ur = ur, phi = phi, uphi = uphi, z_cyl = z_cyl, uz_cyl = uz_cyl)
        
    

el
0.01


/home/petros/anaconda3/lib/python3.7/site-packages/scipy/integrate/odepack.py:236: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)


runtime for gamma0 = 665 is 0h 0m 4s
runtime for gamma0 = 739 is 0h 0m 5s
runtime for gamma0 = 888 is 0h 0m 6s
runtime for gamma0 = 746 is 0h 0m 1s
runtime for gamma0 = 733 is 0h 0m 7s
runtime for gamma0 = 211 is 0h 0m 27s
runtime for gamma0 = 829 is 0h 0m 15s
runtime for gamma0 = 643 is 0h 0m 4s
runtime for gamma0 = 248 is 0h 0m 28s
total runtime = 0h 1m 42s
0.12
runtime for gamma0 = 665 is 0h 0m 19s
runtime for gamma0 = 739 is 0h 0m 13s
runtime for gamma0 = 888 is 0h 0m 12s
runtime for gamma0 = 746 is 0h 0m 16s
runtime for gamma0 = 733 is 0h 0m 14s
runtime for gamma0 = 211 is 0h 0m 14s
runtime for gamma0 = 829 is 0h 0m 18s
runtime for gamma0 = 643 is 0h 0m 14s
runtime for gamma0 = 248 is 0h 0m 33s
total runtime = 0h 2m 37s
0.23
runtime for gamma0 = 665 is 0h 0m 30s
runtime for gamma0 = 739 is 0h 0m 20s
runtime for gamma0 = 888 is 0h 0m 15s
runtime for gamma0 = 746 is 0h 0m 25s
runtime for gamma0 = 733 is 0h 0m 25s
runtime for gamma0 = 211 is 0h 0m 22s
runtime for gamma0 = 829 is 0h 0

In [5]:
dir_name = 'data7'
#Έκχώρηση των παραμέτρων της προσωμοίωσης
with open('{}/par_dic.txt'.format(dir_name), 'rb') as par_dic:
    pdic = pickle.loads(par_dic.read())
k = pdic['multiplicity']
name = pdic['pulsar_name']
pulsar = Pulsars(k)[name]

gamma1 = 1.
B_0 = pdic['char magn field (Blc)']
omegaB = (e_charge*B_0/(e_mass*c))    #γυροσυχνότητα
Rlc = pdic['Radius of lc']
Delta = pdic['diss zone length']

gamma0 = pdic['initial lorentz factors']
init_pos = pdic['initial injection positions']

delta_init = pdic['diss zone width']    #πάχος του φύλλου ρεύματος, αδιάστατο, σε μονάδες [c/ωΒ]



T = pdic['runtime']
N = pdic['number of t points']
t = np.linspace(0.0, T*Delta, N)




for q in [-1, 1]:
    if q == +1:
        charge = 'pos'
    else:
        charge = 'el'
    for w in range(len(init_pos)):

        vec = np.load('{}/cyl_{}_{}_raw.npz'.format(dir_name, int(100*init_pos[w]), charge))
        r, ur, phi, uphi, z_cyl, uz_cyl = vec['r'], vec['ur'], vec['phi'], vec['uphi'], vec['z_cyl'], vec['uz_cyl']
        for i in range(len(gamma0) - 1):
            for j in range(len(r[i])):
                if r[i][j] < Rlc - 15*Delta or r[i][j] > Rlc + 15*Delta:
    #                     print(i, j, r[i][j])
                    r[i][j] = np.NaN
                    phi[i][j] = np.NaN
                    z_cyl[i][j] = np.NaN
                    ur[i][j] = np.NaN
                    uphi[i][j] = np.NaN
                    uz_cyl[i][j] = np.NaN
#                 if z_cyl[i][j] > 10000*delta_init or z_cyl[i][j] < -10000*delta_init:
#                     r[i][j] = np.NaN
#                     phi[i][j] = np.NaN
#                     z_cyl[i][j] = np.NaN
#                     ur[i][j] = np.NaN
#                     uphi[i][j] = np.NaN
#                     uz_cyl[i][j] = np.NaN
        with open('{}/cyl_{}_{}.npz'.format(dir_name, int(100*init_pos[w]), charge), 'wb') as f:       
            np.savez(f, r = r, ur = ur, phi = phi, uphi = uphi, z_cyl = z_cyl, uz_cyl = uz_cyl)

In [7]:
#υπολογισμός ακτίνας καμπυλότητας, κρίσιμης συχνότητας και απωλειών

ss = reload(ss)
for q in [-1, 1]:
    if q == +1:
        charge = 'pos'
    else:
        charge = 'el'
    
    for w in range(0, len(init_pos)):
#     for w in range(4, 5):
        if not (w == 7):
            a = np.load('{}/cyl_{}_{}.npz'.format(dir_name, int(100*init_pos[w]), charge))
            r, ur, phi, uphi, z_cyl, uz_cyl = a['r'], a['ur'], a['phi'], a['uphi'], a['z_cyl'], a['uz_cyl']

            r_curv, nu_crit, p_rad = ss.rc_nc_pr(r, phi, z_cyl, ur, uphi, uz_cyl, Rlc, Delta, delta_init, gamma0, B_0)

            with open('{}/rc_nc_pr_{}_{}.npz'.format(dir_name, int(100*init_pos[w]), charge), 'wb') as f2:
                np.savez(f2, r_curv = r_curv, nu_crit = nu_crit,  p_rad = p_rad)

            en, ph_num, ph_num_out, ph_num_out2, ph_num_sep, ph_en, ph_en_out, ph_en_out2, ph_en_sep, ph_num_tot, ph_en_tot = ss.spectrum(r, z_cyl, 
                                                                                                                                          nu_crit, 
                                                                                                                                          p_rad, 
                                                                                                                                          gamma0, 
                                                                                                                                          Rlc, 
                                                                                                                                          Delta, 
                                                                                                                                          delta_init, 
                                                                                                                                          t)

            with open('{}/spectrum_{}_{}.npz'.format(dir_name, int(100*init_pos[w]), charge), 'wb') as f3:
                np.savez(f3, en = en, ph_num = ph_num, ph_num_out = ph_num_out, ph_num_out2 = ph_num_out2, 
                         ph_num_sep = ph_num_sep, 
                         ph_en = ph_en, ph_en_out = ph_en_out, ph_en_out2 = ph_en_out2, ph_en_sep = ph_en_sep,
                         ph_num_tot = ph_num_tot, ph_en_tot = ph_en_tot)

/home/petros/Desktop/diplomatiki/git_python_scripts/speiser_orbit/speiser_fun_cyl.py:10: RuntimeWarning: overflow encountered in double_scalars
  g = np.sqrt(1 + u1**2 + u2**2 + u3**2)
/home/petros/Desktop/diplomatiki/git_python_scripts/speiser_orbit/speiser_spec.py:21: RuntimeWarning: overflow encountered in double_scalars
  + uphi[i][j]**2/(r[i][j]*sfc.gamma(ur[i][j], uphi[i][j], uz_cyl[i][j])) - Frad[-1]*ur[i][j]*0
/home/petros/Desktop/diplomatiki/git_python_scripts/speiser_orbit/speiser_spec.py:21: RuntimeWarning: invalid value encountered in double_scalars
  + uphi[i][j]**2/(r[i][j]*sfc.gamma(ur[i][j], uphi[i][j], uz_cyl[i][j])) - Frad[-1]*ur[i][j]*0
/home/petros/Desktop/diplomatiki/git_python_scripts/speiser_orbit/speiser_fun_cyl.py:93: RuntimeWarning: overflow encountered in double_scalars
  metro_u = (u1**2 + u2**2 + u3**2)**0.5
/home/petros/Desktop/diplomatiki/git_python_scripts/speiser_orbit/speiser_fun_cyl.py:111: RuntimeWarning: invalid value encountered in double_scalars
 

In [7]:
# %matplotlib notebook
# plt.plot(h*nu_crit[1])

In [9]:
dic_cyl

{'hu': array([ 1.99971420e-003,  4.67154730e-310,  0.00000000e+000, ...,
        -2.10479023e+011, -3.65194812e-300,  3.95276232e-318]),
 'tcur': array([ 7.50518905e+000,  6.92847760e-310, -1.14132277e+013, ...,
         6.92845247e-310,  6.92845247e-310,  3.95276232e-318]),
 'tolsf': array([2.47032823e-323, 4.67154552e-310, 4.67154600e-310, ...,
        4.42869941e+010, 4.42874370e+010, 3.95252517e-318]),
 'tsw': array([ 0.00000000e+000,  0.00000000e+000,  6.07229900e-318, ...,
        -7.75876015e-257, -1.99259106e-136,  3.95268327e-318]),
 'nst': array([       3000,       32650,  -532978992, ..., -1521830208,
              32650, -1521830184], dtype=int32),
 'nfe': array([       7495,  1086210356,          49, ..., -1031552128,
        -1031617604, -2084352854], dtype=int32),
 'nje': array([         0,      32650, -570197200, ...,   30918801,   30919235,
          30919805], dtype=int32),
 'nqu': array([          1,       32650,  -515002432, ..., -1528401752,
              32650, -1